# Sweep

<img style="float: right" src="img/1-title.gif" width="600">

Sweeps sind eine beliebte Methode zur Messung der Übertragungsfunktion eines linearen Systems, da sie eine Reihe von positiven Eigenschaften aufweisen wie bspw. einen (einstellbaren) glatten Amplitudenfrequenzgang.

## Inhalt  
<table style="width:256px; border: 1px solid black; display: inline-block">
    <tr>
        <td style="text-align:right" width=64px><img src="img/1-1.jpg" style="float:left"></td>
        <td style="text-align:left" width=256px>
            <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#1'>
                1. Zeitsignal
            </a>
        </td>
    </tr>  
    <tr>
        <td style="text-align:right"><img src="img/1-2.jpg" style="float:left"></td>
        <td style="text-align:left" width=128px>
            <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#2'>
                2. Spektrum
            </a>
        </td>
    </tr>
</table>

----

<a id='1'></a>
<div>
    <img src="img/1-1.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        1. Zeitsignal 
    </h2>
</div>

Ein Sweep $x(t)$ im Zeitbereich ist eine harmonische Schwingung mit zeitabhängigem Phasenargument $\varphi (t)$, dessen Momentanfrequenz $\omega (t)$ mit der Zeit monoton zu- oder abnimmt (weswegen er auch Gleitsinus genannt wird, in Abgrenzung zu Sinusschwingungen mit einer konstanten Phase) :
\begin{equation*}
x(t)=x_{0}\sin (\varphi (t))\; \; mit \; \; \omega (t)=\frac{\mathrm{d} \varphi (t)}{\mathrm{d} t}\; \; 
\rightarrow \; \;  x(t)=x_{0}\sin (\int \omega (t)dt + \varphi_{0})
\end{equation*}  
In der Umgebung eines Zeitpunkts $t_{0}$ mit der _Momentanfrequenz_ $\omega _{0}$ nähert sich $x(t)$ einer Sinusfunktion mit der Frequenz $\omega _{0}$ an.

Im Nachfolgenden soll der in Übung 6.1 händisch berechnete Sweep implementiert und visualisiert werden. Am Ende wird, wie bei den Filtern in Thema1, die Möglichkeit der Verwendung der [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html) toolbox erwähnt.

Zunächst werden die Start- und Endfrequenz $f_{start}$ und $f_{end}$ bzw. $\omega_{start}$ und $\omega_{end}$ sowie die notwendige Abtastfrequenz $f_s$ festgelegt:

In [ ]:
'''
Module importieren und Grundparameter definieren:
'''
import numpy as np

fs_Hz = 16e3     # Abtastfrequenz (für diese Aufgabe relativ beliebig)
dt_s = 1/fs_Hz   # Zeitintervall
fStart_Hz = 200  # Startfrequenz [Hz]
fEnd_Hz = 1000   # Endfrequenz [Hz]
wStart_rad_per_s = fStart_Hz * 2 * np.pi  # Startfrequenz [rad]
wEnd_rad_per_s = fEnd_Hz * 2 * np.pi  # Endfrequenz [rad]
wm_rad_per_s = (wStart_rad_per_s + wEnd_rad_per_s) / 2  # Mittenfrequenz [rad/s]

Anschließend werden die Funktionen für die Phasenwinkel $\varphi_1(t)$ und $\varphi_2(t)$ definiert:

In [ ]:
def calculatePhi1(t_s, wStart_rad_per_s, k, T_s):
    phi = k*t_s**4/(3*T_s**2) + wStart_rad_per_s*t_s
    return phi

In [ ]:
def calculatePhi2(t_s, wStart_rad_per_s, k, T_s):
    phi = k * t_s**4 / (3 * T_s**2) - 4*k / (3*T_s) * t_s**3 \
    + 2*k * t_s**2 - k*T_s*t_s + wStart_rad_per_s*t_s \
    + 7*k/48 * T_s**2 - wStart_rad_per_s*T_s/2
    return phi

Durch die Diskontinuität in der Einhüllenden ($a(t)$) müssen in diesem Spezialfall zusätzliche Bedingungen erfüllt werden, damit die Phase an der Übergangssstelle kontinuierlich verläuft. Zum einen muss der Sweep-Phasenwinkel $\varphi(t)$ an der Stelle $T/2$ gleich sein:
\begin{equation}
\sin\left(\varphi_1\left(\frac{T}{2}\right)\right) = \sin\left(\varphi_2\left(\frac{T}{2}\right)\right) 
\end{equation}

Substituiert man hier die berechneten Gleichungen aus der Übung für $\varphi_1(T/2)$ und $\varphi_2(T/2)$ ergibt sich die erste Bedingung
\begin{equation}
\sin\left(\frac{T\cdot(7\omega_1 + \omega_2)}{16}\right) = \sin\left(0\right) = 0.
\end{equation}

Zum anderen muss auch die Ableitung des Sweep-Phasenwinkels, $\frac{\mathrm{d}\varphi(t)}{\mathrm{d}t}$ gleich sein, d.h. aus der ersten Bedingung folgt (innere Ableitung nicht vergessen):

\begin{align}
&\cos\left(\frac{T\cdot(7\omega_1 + \omega_2)}{16}\right)\cdot \left(\frac{\omega_1 + \omega_2}{2}\right) = \cos\left(0\right)\cdot \left(\frac{\omega_1 + \omega_2}{2}\right) = 1 \\
&\Leftrightarrow \cos\left(\frac{T\cdot(7\omega_1 + \omega_2)}{16}\right) = 1.
\end{align}

Beide Gleichungen sind erfüllt, wenn das Argument ein Vielfaches von $2\pi$ ist. Damit kann die Sweepdauer $T$ explizit durch

\begin{equation}
\frac{T}{16}\left(\omega_1 + \omega_2\right) = 2\pi\cdot n,\qquad n=0,1,2,3,\dots
\end{equation}

berechnet werden und ist durch $n\in \mathbb{N}$ nicht komplett frei wählbar.
Dieser Zusammenhang für die Periode $T$ ist in der nachfolgenden Funktion definiert:

In [ ]:
# Phasenwinkelfunktion
def calculatePeriod(wStart_rad_per_s, wEnd_rad_per_s, n_periods):
    T_s = 2*n_periods*np.pi*16 / (7*wStart_rad_per_s + wEnd_rad_per_s)  
    return T_s

Zu guter Letzt werden noch die beiden Funktionen für die Berechnung der Amplitudenmodulaton $a(t)$ für beide Hälften definiert:

In [ ]:
# Amplitudenmodulationsfunktionen:
def calculateAmplitude1(t_s, T_s):
    a = 2/T_s*t_s
    return a

def calculateAmplitude2(t_s, T_s):
    a = -2/T_s*(t_s-T_s)
    return a

In [ ]:

# Sweepdauer T berechnen in Abhängigkeit von ganzen Perioden 2*pi*n:
n_periods = 300;
T_s = calculatePeriod(wStart_rad_per_s, wEnd_rad_per_s, n_periods)  # die Dauer des Sweeps
print("Sweepdauer: {} Sekunden.\n".format(T_s))

# Konstante k:
k = 3/T_s * (wEnd_rad_per_s - wStart_rad_per_s)  

Jetzt können wir den Sweep damit berechnen und plotten:

In [ ]:
'''
Aufgabe: Inplementieren des Sweeps nach Übung 6.1
'''
import matplotlib.pyplot as plt

# Laufvariable t definieren:
t1_s = np.linspace(0, T_s/2, int(fs_Hz*T_s/2))
t2_s = np.linspace(T_s/2+dt_s, T_s, int(fs_Hz*T_s/2-1))

# Phasenwinkel für beide Abschnitte:
phi1_rad = calculatePhi1(t1_s, wStart_rad_per_s, k, T_s)
phi2_rad = calculatePhi2(t2_s, wStart_rad_per_s, k, T_s)

# Einhüllende für beide Abschnitte:
a1 = calculateAmplitude1(t1_s, T_s)
a2 = calculateAmplitude2(t2_s, T_s)

# Vollständigen Sweep zusammensetzen:
sweep1 = a1 * np.sin(phi1_rad)
sweep2 = a2 * np.sin(phi2_rad)
t_s = np.append(t1_s, t2_s)        
sweep = np.append(sweep1, sweep2)

# Plot
plt.title('Sweep von %d Hz bis %d Hz' %(fStart_Hz, fEnd_Hz))
plt.xlabel('Zeit [s]') 
plt.ylabel('x(t)') 
plt.plot(t_s, sweep)
plt.gcf().set_size_inches(20, 5)
plt.show()

----

<a id='2'></a>
<div>
    <img src="img/1-2.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        2. Spektrum 
    </h2>
</div>

Um sicherzustellen, dass der Sweep tatsächlich das (annähernd) glatte Spektrum über den definierten Frequenzbereich von $f_{start}$ bis $f_{end}$ besitzt, wird der Amplitudenfrequenzgang $|X(k)|$ über die FFT berechnet:

In [ ]:
'''
Aufgabe: Spektrum des Sweeps berechnen
'''
from scipy import fftpack

# Parameter
N = int(2**(np.floor(np.log2(len(sweep))))) # Länge von FFT
f_Hz = np.linspace(0, fs_Hz/2, int(N/2))    # Frequenzbereich

# FFT
sweep_fft = fftpack.fft(sweep, N)
sweep_fft_plot = np.abs(sweep_fft[:len(f_Hz)]) / int(N/2)

# Plot 
plt.title('Amplitudenfrequenzgang')
plt.xlabel('Frequenz [Hz]') 
plt.ylabel('|X(f)|') 
plt.plot(f_Hz, sweep_fft_plot)
plt.axis([fStart_Hz*0.5, fEnd_Hz*1, 0, np.max(sweep_fft_plot)*1.1])
plt.gcf().set_size_inches(10, 5)
plt.show()

Abspielen des Sweepssignal $x(t)$ ergibt das charakteristische Sweepgeräusch:

In [ ]:
'''
Aufgabe: Sweep abspielen
'''
import numpy as np
import simpleaudio as sa

# Werte in 16-Bit-Daten konvertieren
sound = (sweep*(2**15-1)/np.max(np.abs(sweep))).astype(np.int16)

# Abspielen
play_obj = sa.play_buffer(sound, 1, 2, int(fs_Hz))
play_obj.wait_done()

----

<a id='2'></a>
<div>
    <h2 style="position: relative; top: 6px; left: 6px">
        *. Erweiterung 
    </h2>
</div>

Im Modul scipy ist die Funktion [signal.chirp()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.chirp.html) als Frequenzgesteuerter Kosinusgenerator vorhanden, dadurch können Sweeps deutlich einfacher erzeugt werden:

In [ ]:
'''
Beispiel: Sweep mittels signal.chirp() erzeugen
'''
from scipy.signal import chirp, spectrogram
import matplotlib.pyplot as plt
import numpy as np
import simpleaudio as sa

# Sweep-Parameter definieren:
fsChirp_Hz = 16000
TChirp_s = 3
tChirp_s = np.linspace(0, TChirp_s, TChirp_s*fsChirp_Hz, endpoint=False)
chirpStartFreq_Hz = 200
chirpEndFreq_Hz = 1000

# Sweep und dessen Amplitudenfrequeuzgang berechnen:
xSweep = chirp(tChirp_s, chirpStartFreq_Hz, TChirp_s, chirpEndFreq_Hz, 'linear')
fChirp_Hz, tChirpSg_s, XSweep_f = spectrogram(xSweep, fsChirp_Hz, nperseg=250)

# Plot
plt.subplot(121)
plt.title('Linearer Chirp, f(0)=%d Hz, f(%d)=%d Hz' %(chirpStartFreq_Hz, TChirp_s, chirpEndFreq_Hz))
plt.xlabel('Zeit [s]') 
plt.ylabel('$x_{sweep}$(t)') 
plt.plot(tChirp_s, xSweep)
plt.subplot(122)
plt.title('Spektrogramm')
plt.xlabel('Frequenz [Hz]') 
plt.ylabel('|$X_{sweep}$(f)|') 
plt.pcolormesh(tChirpSg_s, fChirp_Hz, XSweep_f,shading='gouraud')
plt.gcf().set_size_inches(15, 5)
plt.show()

In [ ]:
'''
Aufgabe: Sweep abspielen
'''
import numpy as np
import simpleaudio as sa

# Werte in 16-Bit-Daten konvertieren
sound = (xSweep*(2**15-1)/np.max(np.abs(xSweep))).astype(np.int16)

# Abspielen
play_obj = sa.play_buffer(sound, 1, 2, int(fsChirp_Hz))
play_obj.wait_done()

Die Verwendung eines Spektrogramms statt eines einfachen Spektrums zur Darstellung des Frequenzbereiches (in diesem Falle in zusätzlicher Abhängigkeit von der Zeit) wird im nächsten Notebook noch ausführlich behandelt.

----

### References

1. Titelbild von [wikimedia](https://commons.wikimedia.org/wiki/File:Chirp_animation.gif?uselang=de)  
2. [Sweep (Signalverarbeitung)](https://de.wikipedia.org/wiki/Sweep_(Signalverarbeitung))  
3. [Sinusoidal Sweep Signals](https://learn.digilentinc.com/Documents/132)  
4. [Sine Sweep](https://theaudioprogrammer.com/signal-analysis-ii-linear-vs-logarithmic-sine-sweep/)  